In [1]:
import datasets
import pandas as pd

In [5]:
df_test = pd.read_csv("/kaggle/input/nlp-ua-locations-extractions/test.csv")

In [6]:
!pip install lingua-language-detector --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 MB 11.3 MB/s eta 0:00:0000:0100:01


In [7]:
from lingua import Language, LanguageDetectorBuilder
languages = [Language.UKRAINIAN, Language.RUSSIAN]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

In [8]:
def contains_uk_letters(input_string):
    ukrainian_letters = "їієґ"
    ukrainian_letters_uppercase = ukrainian_letters.upper()
    pattern = re.compile(f"[{ukrainian_letters}{ukrainian_letters_uppercase}]")
    return bool(pattern.search(input_string))
def contains_ru_letters(input_string):
    ru_letters = "ыъэё"
    ru_letters_uppercase = ru_letters.upper()
    pattern = re.compile(f"[{ru_letters}{ru_letters_uppercase}]")
    return bool(pattern.search(input_string))
import re

def detect_lang(text):
    if contains_uk_letters(text):
        return 'uk'
    elif contains_ru_letters(text):
        return 'ru'
    else:
        return 'uk' if detector.detect_language_of(text) == Language.UKRAINIAN else 'ru'

In [9]:
df_test['lang']  = df_test['text'].apply(detect_lang)


In [10]:
from transformers import pipeline

model_checkpoint_uk = "/kaggle/input/uk-ner-multibert/bert-ua-loc-ner/checkpoint-14096/"
token_classifier_uk = pipeline(
    "token-classification", model=model_checkpoint_uk, aggregation_strategy="simple"
)

model_checkpoint_ru = "/kaggle/input/fork-of-uk-ner-multibert/multibert-ner-ru/checkpoint-12500/"
token_classifier_ru = pipeline(
    "token-classification", model=model_checkpoint_ru, aggregation_strategy="simple"
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [11]:
df_test.loc[1, "text"]

'🥤В Києві за 91,13 млн гривень починаються роботи зі знесення будівлі цеху №5 заводу “Більшовик”, які необхідні для продовження реконструкції “Шулявського шляхопроводу”.\n\n🤝Договір на проведення цих робіт КП "Дирекція будівництва шляхово-транспортних споруд Києва" уклало з ТОВ “Мостицький-2”, відхиливши вигіднішу пропозицію в тендері.\n\n🧐“Мостицький-2” відоме як постійний генпідрядник будівництва скандальних житлових комплексів Edelburg Development, яке пов\'язують з сином екс-мера столиці Леоніда Черновецького Степаном👤. \n\nЦі компанії тісно сплетені бізнесовими та навіть родинними нитями👨\u200d👩\u200d👧\u200d👦.\n🧶Водночас зв\'язки обраного для знесення цеху підрядника ведуть до ще однієї одіозної фігури - сумнозвісного столичного забудовника Максима Микитася, компанія з орбіти якого майже за 1,5 млрд гривень власне займається всією реконструкцією Шулявського шляхопроводу🏗'

In [14]:
token_classifier_uk(df_test.loc[1, "text"])

[{'entity_group': 'LOC',
  'score': 0.99292105,
  'word': 'Києві',
  'start': 3,
  'end': 8},
 {'entity_group': 'LOC',
  'score': 0.9708286,
  'word': 'Шулявського шляхопроводу',
  'start': 142,
  'end': 166},
 {'entity_group': 'LOC',
  'score': 0.940351,
  'word': 'Шулявського',
  'start': 838,
  'end': 849}]

In [15]:
import emoji

In [18]:
def contains_emoji(text):
    for char in text:
        if emoji.is_emoji(char):
            return True
    else:
        return False

def extract_locations(text, th_uk=0.93, th_ru=0.93):    
    lang = detect_lang(text)
   
    if lang == 'uk':
        ents = token_classifier_uk(text)
        res = []
        for ent in ents:
            if ent['score'] >= th_uk and "#" not in ent['word'] and not contains_emoji(ent['word']):
                word = re.sub(r"\s*([ʼ\'’-])\s*", r"\1", ent['word'])
                res.append(word)
    else:
        ents = token_classifier_ru(text)
        res = []
        for ent in ents:
            if ent['score'] >= th_ru and "#" not in ent['word'] and not contains_emoji(ent['word']):
                word = re.sub(r"\s*([ʼ\'’-])\s*", r"\1", ent['word'])
                res.append(word)
    return res

In [19]:
df_test.loc[106:107, "text"].apply(extract_locations)

106               [Литві, Польщі, Вільнюс, Люблін]
107    [Україні, просп. Берестейський, 25-А, Київ]
Name: text, dtype: object

In [20]:
df_test.loc[106:107, "text"].apply(extract_locations)[107][1] == "просп. Берестейський, 25-А"

True

In [21]:
sample_pred = df_sample.loc[15, 'locations_pred'][0]
sample_true = df_sample.loc[15, 'locations'][0]

NameError: name 'df_sample' is not defined

In [22]:
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
df_test['locations'] = df_test['text'].progress_apply(extract_locations)

  0%|          | 0/477 [00:00<?, ?it/s]

In [ ]:
df_test[['text_id', 'locations']].to_csv('multibert_ukru_lingua.csv', index=False)

In [ ]:
df_test

In [ ]:
df_sample = pd.read_csv("/kaggle/input/nlp-ua-locations-extractions/labeling_sample.csv", converters={"locations": eval})

In [ ]:
df_sample['locations_pred'] = df_sample['text'].progress_apply(extract_locations)

In [ ]:
df_sample.loc[15:16, "text"].apply(extract_locations)

In [ ]:
df_sample[['text', 'locations', 'locations_pred']]

In [ ]:
df_sample.loc[15]

In [ ]:
from typing import List
import re

def hard_processing(text):
    text = re.sub(r"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\b\w\b\s?', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def comp_metric(y_true: List[List[str]], y_pred: List[List[str]]):
    assert len(y_true) == len(y_pred)
    tp, fp, fn = 0.0, 0.0, 0.0

    for y_true_sample, y_pred_sample in zip(y_true, y_pred):
        y_true_sample = set([hard_processing(s) for s in y_true_sample])
        y_pred_sample = set([hard_processing(s) for s in y_pred_sample])
        tp += len(set(y_true_sample) & set(y_pred_sample))
        fp += len(set(y_pred_sample) - set(y_true_sample))
        fn += len(set(y_true_sample) - set(y_pred_sample))
    
    precision = tp / (tp + fp) if tp + fp != 0 else 0.0 if tp + fn != 0.0 else 1.0
    recall = tp / (tp + fn) if tp + fn != 0 else 1.0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0.0

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
comp_metric(df_sample.loc[14:16, 'locations'], df_sample.loc[14:16, 'locations_pred'])

In [ ]:
res = pd.read_csv("/kaggle/working/multibert_w_langd_apstr.csv", converters={"locations": eval})

In [ ]:
df_test.loc[9:10, "text"].apply(extract_locations)

In [ ]:
res.loc[9][0]